# ngvs_intro.ipynb

### Isaac Cheng - September 2021

Just exploring the NGVS data (of VCC 792 aka NGC 4380) given to me. Read [this webpage](http://www.cadc-ccda.hia-iha.nrc-cnrc.gc.ca/en/community/ngvs/docs/ngvsdoc.html) for lots of useful information! ALso look at [this PDF](https://www.ucolick.org/~bolte/AY257/s_n.pdf) for a useful overview of CCD signal/noise satistics. Also read the section on binning.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib.colors import LogNorm
import matplotlib.font_manager as fm
from mpl_toolkits.axes_grid1.anchored_artists import AnchoredSizeBar
from astropy.wcs.utils import proj_plane_pixel_scales
import astropy.units as u
from astropy.io import fits
from astropy.wcs import WCS
import copy
%matplotlib widget

In [2]:
def shiftedColorMap(cmap, start=0, midpoint=0.5, stop=1.0, name='shiftedcmap'):
    '''
    Function to offset the "center" of a colormap. Useful for
    data with a negative min and positive max and you want the
    middle of the colormap's dynamic range to be at zero.
    
    From https://stackoverflow.com/a/20528097.

    Input
    -----
      cmap : The matplotlib colormap to be altered
      start : Offset from lowest point in the colormap's range.
          Defaults to 0.0 (no lower offset). Should be between
          0.0 and `midpoint`.
      midpoint : The new center of the colormap. Defaults to 
          0.5 (no shift). Should be between 0.0 and 1.0. In
          general, this should be  1 - vmax / (vmax + abs(vmin))
          For example if your data range from -15.0 to +5.0 and
          you want the center of the colormap at 0.0, `midpoint`
          should be set to  1 - 5/(5 + 15)) or 0.75
      stop : Offset from highest point in the colormap's range.
          Defaults to 1.0 (no upper offset). Should be between
          `midpoint` and 1.0.
    '''
    cdict = {
        'red': [],
        'green': [],
        'blue': [],
        'alpha': []
    }

    # regular index to compute the colors
    reg_index = np.linspace(start, stop, 257)

    # shifted index to match the data
    shift_index = np.hstack([
        np.linspace(0.0, midpoint, 128, endpoint=False), 
        np.linspace(midpoint, 1.0, 129, endpoint=True)
    ])

    for ri, si in zip(reg_index, shift_index):
        r, g, b, a = cmap(ri)

        cdict['red'].append((si, r, r))
        cdict['green'].append((si, g, g))
        cdict['blue'].append((si, b, b))
        cdict['alpha'].append((si, a, a))

    newcmap = mpl.colors.LinearSegmentedColormap(name, cdict)
    plt.register_cmap(cmap=newcmap)

    return newcmap


class MidPointLogNorm(LogNorm):
    """
    LogNorm with adjustable midpoint. From https://stackoverflow.com/a/48632237.
    """
    def __init__(self, vmin=None, vmax=None, midpoint=None, clip=False):
        LogNorm.__init__(self,vmin=vmin, vmax=vmax, clip=clip)
        self.midpoint=midpoint
    def __call__(self, value, clip=None):
        result, is_scalar = self.process_value(value)
        x, y = [np.log(self.vmin), np.log(self.midpoint), np.log(self.vmax)], [0, 0.5, 1]
        return np.ma.array(np.interp(np.log(value), x, y), mask=result.mask, copy=False)


In [3]:
def load_img(imgpath, idx=0):
    """
    Retrieves and returns the image data and header from a .fits file
    given a filepath.
    
    Parameters:
      imgpath :: str
        The path to the .fits file
      idx :: int (optional, default: 0)
        The index of the block from which to extract the data
    
    Returns: imgdata, imgheader, imgwcs
      imgdata :: 2D array
        An array with the pixel values from the .fits file
      imgheader :: `astropy.io.fits.header.Header`
        The header to the .fits file
    """
    with fits.open(imgpath) as hdu_list:
        hdu_list.info()
        # Get image data (typically in PRIMARY block)
        imgdata = hdu_list[idx].data  # 2D array
        # Get image header and WCS coordinates
        imgheader = hdu_list[idx].header
        # imgwcs = WCS(hdu_list[0].header)
    return imgdata, imgheader


def add_scalebar(ax, imgwcs, dist, scalebar_factor=1, label="1 kpc", color="k", loc="lower right",
                 size_vertical=0.5, pad=1, fontsize=12, **kwargs):
    """
    Adds a 1 kpc scale bar (by default) to a plot.
    
    Parameters:
      ax :: `matplotlib.axes._subplots.AxesSubplot`
        The matplotlib axis object on which to add the scalebar
      imgwcs :: `astropy.wcs.wcs.WCS`
        The WCS coordinates of the .fits file
      dist :: `astropy.units.quantity.Quantity` scalar
        The distance to the object
      scalebar_factor :: float
        Factor by which to multiply the 1 kpc scale bar.
      label :: str (optional, default: "1 kpc")
        The scale bar label
      color :: str (optional, default: "k")
        Colour of the scale bar and label
      loc :: str or int (optional, default: "lower right")
        The location of the scale bar and label
      size_vertical :: float (optional, default: 0.5)
        Vertical length of the scale bar (in ax units)
      pad :: float or int (optional, default: 1)
        Padding around scale bar and label (in fractions of the font size)
      fontsize :: str or int (optional, default: 12)
        The font size for the label
      **kwargs :: dict (optional)
        Keyworded arguments to pass to `matplotlib.offsetbox.AnchoredOffsetbox`
        
    Returns: ax.add_artist(scalebar)
      ax.add_artist(scalebar) :: `mpl_toolkits.axes_grid1.anchored_artists.AnchoredSizeBar`
        Adds the scale har to the given ax
    """
    arcsec_per_px = (proj_plane_pixel_scales(imgwcs.celestial)[0] * u.deg).to(u.arcsec)
    # Still deciding whether to use arctan for the next line
    arcsec_per_kpc = np.rad2deg(1 * u.kpc / dist.to(u.kpc) * u.rad).to(u.arcsec)
    px_per_kpc = arcsec_per_kpc / arcsec_per_px
    # Set font properties
    fontproperties = fm.FontProperties(size=fontsize)
    # Add scale bar
    scalebar = AnchoredSizeBar(ax.transData, px_per_kpc * scalebar_factor, label=label, loc=loc,
                               fontproperties=fontproperties, pad=pad, color=color,
                               frameon=False, size_vertical=size_vertical, **kwargs)
    return ax.add_artist(scalebar)


def plot_img_log(imgdata, imgheader, dist,
                 levels=18, cmap="magma", vmin=None, vmax=None, midpoint=None, interp="none",
                 scale_lbl="1 kpc", scale_loc="lower right", scale_color="k",
                 scalebar_factor=1, scale_size_vertical=0.5, scale_pad=1, scale_fontsize=12, 
                 fig_title=None, fig_yax=None, fig_xax=None, fig_cbar=None,
                 figname=None, dpi=300, **imshow_kwargs):
    """
    Plots an image with a spatial scale bar using `imshow` + a log_10 colour bar scale.
    Also optionally saves the figure if given a figname.
    May not use in the future since it is so clunky.
    
    FIXME: add **scale_kwargs
    
    Parameters:
      imgdata :: 2D array
        An array with the pixel values from the .fits file
      imgheader :: `astropy.io.fits.header.Header`
        The header to the .fits file
      dist :: `astropy.units.quantity.Quantity` scalar
        The distance to the object
      levels :: int (optional, default: 18. NOT USED)
        Number of contour levels to use (NOT USED)
      cmap :: str (optional, default: "magma")
        Colour map
      vmin, vmax, midpoint :: float (optional, defaults: None)
        The minimum, maximum, and midpoint of the colour bar. If None,
        use the default values determined by matplotlib
      interp :: str (optional, default: "none")
        Interpolation for `imshow` (e.g., "none", "antiasliased", "bicubic")
      scale_lbl, scale_loc, scale_color :: str (optional,
                                                defaults: "1 kpc", "lower right", "k")
        The label, location, and colour of the scale bar
      scalebar_factor :: float (optional, default: 1)
        Factor by which to multiply the 1 kpc scale bar.
      scale_size_vertical :: float (optional, default: 0.5)
        Vertical length of the scale bar (in axis units)
      scale_pad :: float or int (optional, default: 1)
        Padding around scale bar and label (in fractions of the font size)
      scale_fontsize :: str or int (optional, default: 12)
        The font size for the scale bar label
      fig_title, fig_yax, fig_xax, fig_cbar :: str (optional, defaults: None)
        The title of the plot, y-axis label, x-axis label, and colourbar label.
        If None, do not write labels (though astropy may automatically add coordinate labels)
      figname :: str (optional, default: None)
        Name of the file to save (e.g. "img.pdf", "img_2.png"). By default, it will save the file
        in the directory from which the Python file is called or where the iPython notebook is located.
        If None, do not save the image.
      dpi :: int (optional, default: 300)
        The dpi of the saved image. Relevant for raster graphics only (i.e., not PDFs).
      **imshow_kwargs :: dict (optional)
          Keyworded arguments to pass to `matplotlib.image.AxesImage`
    
      Returns: None
    """
    imgwcs = WCS(imgheader)
    # Remove background pixels
#     imgdata[imgdata <= 0] = np.nan  # or not. May look bad
#     imgdata[imgdata <= 0] = np.min(imgdata[imgdata > 0])
    imgdata[imgdata <= 0] = 1e-15
#     cmap = mpl.cm.get_cmap(cmap, levels)
    cmap = copy.copy(mpl.cm.get_cmap(cmap))
    # Set NaN pixels to lowest colour in colormap
    cmap.set_bad(cmap(0))
    #
    fig, ax = plt.subplots(subplot_kw={"projection": imgwcs})
    if midpoint is None:
        img = ax.imshow(imgdata, cmap=cmap,
                        norm=LogNorm(vmin=vmin, vmax=vmax),
                        interpolation=interp, **imshow_kwargs)
    elif midpoint > 0:
        img = ax.imshow(imgdata, cmap=cmap,
                        norm=MidPointLogNorm(vmin=vmin, vmax=vmax, midpoint=midpoint),
                        interpolation=interp, **imshow_kwargs)
    else:
        raise ValueError("midpoint should be >0 and between vmin & vmax")
    add_scalebar(ax, imgwcs, dist, scalebar_factor=scalebar_factor, label=scale_lbl,
                 color=scale_color, loc=scale_loc, size_vertical=scale_size_vertical,
                 pad=scale_pad, fontsize=scale_fontsize)
    cbar = fig.colorbar(img)
    cbar.ax.tick_params(which="both", direction="out")
    # cbar.ax.minorticks_off()
    cbar.set_label(fig_cbar) if fig_cbar is not None else None
    ax.tick_params(which="both", direction="out")
    ax.set_xlabel(fig_xax) if fig_xax is not None else None
    ax.set_ylabel(fig_yax) if fig_yax is not None else None
    ax.set_title(fig_title) if fig_title is not None else None
    ax.grid(False)
    ax.set_aspect("equal")
    fig.savefig(figname, bbox_inches="tight", dpi=dpi) if figname is not None else None
    plt.show()


def plot_img_linear(imgdata, imgheader, dist,
                    levels=18, cmap="magma", vmin=None, vmax=None, midpoint=0.5, interp="none",
                    scale_lbl="1 kpc", scale_loc="lower right", scale_color="k",
                    scalebar_factor=1, scale_size_vertical=0.5, scale_pad=1, scale_fontsize=12, 
                    fig_title=None, fig_yax=None, fig_xax=None, fig_cbar=None,
                    figname=None, dpi=300, **imshow_kwargs):
    """
    Plots an image with a spatial scale bar using `imshow` + a linear colour bar scale.
    Also optionally saves the figure if given a figname.
    May not use in the future since it is so clunky.
    
    FIXME: add **scale_kwargs
    
    Parameters:
      imgdata :: 2D array
        An array with the pixel values from the .fits file
      imgheader :: `astropy.io.fits.header.Header`
        The header to the .fits file
      dist :: `astropy.units.quantity.Quantity` scalar
        The distance to the object
      levels :: int (optional, default: 18. NOT USED)
        Number of contour levels to use (NOT USED)
      cmap :: str (optional, default: "magma")
        Colour map
      vmin, vmax, midpoint :: float (optional, defaults: None)
        The minimum, maximum of the colour bar. If None,
        use the default values determined by matplotlib
      midpoint :: float (optional, default: 0.5)
        The midpoint of the colourbar. Must be between 0.0 and 1.0.
      interp :: str (optional, default: "none")
        Interpolation for `imshow` (e.g., "none", "antiasliased", "bicubic")
      scale_lbl, scale_loc, scale_color :: str (optional,
                                                defaults: "1 kpc", "lower right", "k")
        The label, location, and colour of the scale bar
      scalebar_factor :: float (optional, default: 1)
        Factor by which to multiply the 1 kpc scale bar.
      scale_size_vertical :: float (optional, default: 0.5)
        Vertical length of the scale bar (in axis units)
      scale_pad :: float or int (optional, default: 1)
        Padding around scale bar and label (in fractions of the font size)
      scale_fontsize :: str or int (optional, default: 12)
        The font size for the scale bar label
      fig_title, fig_yax, fig_xax, fig_cbar :: str (optional, defaults: None)
        The title of the plot, y-axis label, x-axis label, and colourbar label.
        If None, do not write labels (though astropy may automatically add coordinate labels)
      figname :: str (optional, default: None)
        Name of the file to save (e.g. "img.pdf", "img_2.png"). By default, it will save the file
        in the directory from which the Python file is called or where the iPython notebook is located.
        If None, do not save the image.
      dpi :: int (optional, default: 300)
        The dpi of the saved image. Relevant for raster graphics only (i.e., not PDFs).
      **imshow_kwargs :: dict (optional)
          Keyworded arguments to pass to `matplotlib.image.AxesImage`
    
      Returns: None
    """
    imgwcs = WCS(imgheader)
    # Remove background pixels
#     imgdata[imgdata <= 0] = np.nan  # or not. May look bad
#     imgdata[imgdata <= 0] = np.min(imgdata[imgdata > 0])
#     imgdata[imgdata <= 0] = 1e-15
#     cmap = mpl.cm.get_cmap(cmap, levels)
    cmap = mpl.cm.get_cmap(cmap)
    cmap = copy.copy(shiftedColorMap(cmap, midpoint=midpoint))
    # Set NaN pixels to lowest colour in colormap
    cmap.set_bad(cmap(0))
    #
    fig, ax = plt.subplots(subplot_kw={"projection": imgwcs})
    img = ax.imshow(imgdata, cmap=cmap, vmin=vmin, vmax=vmax, **imshow_kwargs)
    add_scalebar(ax, imgwcs, dist, scalebar_factor=scalebar_factor, label=scale_lbl,
                 color=scale_color, loc=scale_loc, size_vertical=scale_size_vertical,
                 pad=scale_pad, fontsize=scale_fontsize)
    cbar = fig.colorbar(img)
    cbar.ax.tick_params(which="both", direction="out")
    # cbar.ax.minorticks_off()
    cbar.set_label(fig_cbar) if fig_cbar is not None else None
    ax.tick_params(which="both", direction="out")
    ax.set_xlabel(fig_xax) if fig_xax is not None else None
    ax.set_ylabel(fig_yax) if fig_yax is not None else None
    ax.set_title(fig_title) if fig_title is not None else None
    ax.grid(False)
    ax.set_aspect("equal")
    fig.savefig(figname, bbox_inches="tight", dpi=dpi) if figname is not None else None
    plt.show()

In [4]:
vcc792_path = "/arc/home/IsaacCheng/coop_f2021/ngvs_data/NGVS-2-2.l.g.Mg004.3136_8588_6905_10184.fits"
vcc792_dist = 16.5 * u.Mpc
data, header = load_img(vcc792_path)
plot_img_log(data, header, vcc792_dist, cmap="gray", vmin=1e-1, midpoint=23, vmax=None,
             scalebar_factor=10, scale_color="w", scale_lbl="10 kpc", fig_title="VCC 792 / NGC 4380",
             fig_yax="Dec (J2000)", fig_xax="RA (J2000)", fig_cbar="Log g-Band Flux", figname="vcc792_g_v1.pdf")

Filename: /arc/home/IsaacCheng/coop_f2021/ngvs_data/NGVS-2-2.l.g.Mg004.3136_8588_6905_10184.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      52   (5453, 3280)   float32   


the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [5]:
vcc792_g_path = "/arc/home/IsaacCheng/coop_f2021/ngvs_data/NGVS-2-2.l.g.Mg004.3136_8588_6905_10184.fits"
vcc792_i_path = "/arc/home/IsaacCheng/coop_f2021/ngvs_data/NGVS-2-2.l.i.Mg004.3136_8588_6905_10184.fits"
data_g, header_g = load_img(vcc792_g_path)
data_i, header_i = load_img(vcc792_i_path)
# # Double-checking that the images have the same extent
# print(WCS(header_g).pixel_to_world(0,0))
# print(WCS(header_i).pixel_to_world(0,0))
# print(WCS(header_g).pixel_to_world(*data_g.shape))
# print(WCS(header_i).pixel_to_world(*data_i.shape))
plot_img_log(data_g, header_g, vcc792_dist, cmap="gray", vmin=1e-1, midpoint=23, vmax=None,
             scalebar_factor=10, scale_color="w", scale_lbl="10 kpc", fig_title="VCC 792 / NGC 4380",
             fig_yax="Dec (J2000)", fig_xax="RA (J2000)", fig_cbar="Log g-Band Flux")
plot_img_log(data_i, header_i, vcc792_dist, cmap="gray", vmin=1e-1, midpoint=23, vmax=None,
             scalebar_factor=10, scale_color="w", scale_lbl="10 kpc", fig_title="VCC 792 / NGC 4380",
             fig_yax="Dec (J2000)", fig_xax="RA (J2000)", fig_cbar="Log i-Band Flux")

Filename: /arc/home/IsaacCheng/coop_f2021/ngvs_data/NGVS-2-2.l.g.Mg004.3136_8588_6905_10184.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      52   (5453, 3280)   float32   
Filename: /arc/home/IsaacCheng/coop_f2021/ngvs_data/NGVS-2-2.l.i.Mg004.3136_8588_6905_10184.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU      52   (5453, 3280)   float32   


the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [6]:
gi_colour = -2.5 * np.log10(data_g / data_i)
# plt.hist(gi_colour.flatten(), bins=50)
# plt.show()
plot_img_linear(gi_colour, header_i, vcc792_dist, cmap="plasma", midpoint=0.5,
#                 vmin=np.percentile(gi_colour.flatten(), 22), vmax=np.percentile(gi_colour.flatten(), 83),
                vmin=0.5, vmax=1.6,
                interp="antialiased",
                scalebar_factor=10, scale_color="w", scale_size_vertical=20, scale_lbl="10 kpc", scale_fontsize=15,
                fig_title="VCC 792 / NGC 4380", fig_yax="Dec (J2000)", fig_xax="RA (J2000)",
                fig_cbar="g-i Colour Index", figname="vcc792_g-i_dpi300_v2.1.png")

the RADECSYS keyword is deprecated, use RADESYSa. [astropy.wcs.wcs]


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [7]:
plt.close("all")